In [ ]:
"""
converts ground truth geojson files (both buildings and roads) to a csv of format: 
The geojsons should be prepped (includes "inferred_speed_mps" as property) and cleaned using the data_prep functions.

'ImageId', 'Object','Wkt_Pix','Flooded', 'length_m', 'travel_time_s'  

"""

import os
import glob
import json

import pandas as pd
import shapely.wkt
from shapely.geometry import shape, Polygon, LineString, MultiLineString
from osgeo import gdal, osr, ogr

def make_wgs84_utm_srs(longitude, latitude):
    """ create a Spatial Reference object that is a WGS84 UTM projected coord system
    from longitude and latitude values."""
    north = int(latitude > 0)
    approx_zone = int((longitude + 180) / 6)
    srs = osr.SpatialReference()
    srs.SetUTM(approx_zone, north) # zone, north=1,
    srs.SetWellKnownGeogCS("WGS84")
    return srs

def lat_lon_to_row_col(xs, ys, geotran):
    cols = []
    rows = []
    for i in range(len(xs)):
        point = ogr.Geometry(ogr.wkbPoint)
        point.AddPoint(xs[i], ys[i])
        newx, newy = point.GetX(), point.GetY()
        col = (newx - geotran[0]) / geotran[1]
        row = (newy - geotran[3]) / geotran[5]
        cols.append(col)
        rows.append(row)
    return cols, rows
    
def length_of_segments(full_linestring_wkt, x_node, y_node):
    source = osr.SpatialReference() # the input dataset is in wgs84
    source.ImportFromEPSG(4326)
    source.SetAxisMappingStrategy(osr.OAMS_TRADITIONAL_GIS_ORDER) # this is required for gdal>=3.0. The axis ordering is different between 3.x (y, x) and 2.x (x, y). see: https://github.com/OSGeo/gdal/issues/1546
    target_srs = make_wgs84_utm_srs(x_node, y_node)
    transform_to_utm = osr.CoordinateTransformation(source, target_srs) # transform from wgs84 gcs to utm
    
    ogr_linestring = ogr.CreateGeometryFromWkt(full_linestring_wkt)
    ogr_linestring.Transform(transform_to_utm)
    
    shapely_linestring = shapely.wkt.loads(ogr_linestring.ExportToWkt())
    
    segments = [LineString([shapely_linestring.coords[i], shapely_linestring.coords[i+1]]) for i in range(len(shapely_linestring.coords) - 1)]
    lengths_m = []
    for s in segments:
        lengths_m.append(s.length)
    transform_to_utm = None
    return lengths_m

def get_records(geojson, image):
    """
    for roads
    """
    f = open(geojson)
    data = json.load(f)
    f.close()
    
    image_id = os.path.basename(image).split(".")[0]
    
    imds = gdal.Open(image) # in espg 4326
    geotran = imds.GetGeoTransform()
    imds = None
    
    out_records = [] # id, Road, 
    for feature in data["features"]:
        geometry = shape(feature["geometry"]) # in epsg 4326
        if geometry.geom_type == "MultiLineString":
            for line_geom in geometry:
                x, y = line_geom.coords.xy

                cols, rows = lat_lon_to_row_col(x, y, geotran)
                segment_lengths_m = length_of_segments(line_geom.wkt, x[0], y[0])

                flood_attr = True
                if feature["properties"]["flooded"] is None:
                    flood_attr = False

                for i in range(len(cols)-1):
                    wktsegment = LineString([(cols[i], rows[i]), (cols[i+1], rows[i+1])]).wkt
                    out_records.append([image_id, "Road", wktsegment, flood_attr,
                                        segment_lengths_m[i],
                                        segment_lengths_m[i] / feature["properties"]["inferred_speed_mps"]])
        else:
            x, y = geometry.coords.xy

            cols, rows = lat_lon_to_row_col(x, y, geotran)
            segment_lengths_m = length_of_segments(geometry.wkt, x[0], y[0])

            flood_attr = True
            if feature["properties"]["flooded"] is None:
                flood_attr = False

            for i in range(len(cols)-1):
                wktsegment = LineString([(cols[i], rows[i]), (cols[i+1], rows[i+1])]).wkt
                out_records.append([image_id, "Road", wktsegment, flood_attr,
                                    segment_lengths_m[i],
                                    segment_lengths_m[i] / feature["properties"]["inferred_speed_mps"]])
    return out_records
    
def match_im_label(annotations, pre_images, post_images):
    out_pre = []
    out_anno = []
    out_post = []
    for i in annotations:
        tileid = os.path.basename(i).split(".")[0]
        pre_im = [j for j in pre_images if f"_{tileid}.tif" in j][0]
        post_im = [j for j in post_images if f"_{tileid}.tif" in j][0]
        
        out_anno.append(i)
        out_pre.append(pre_im)
        out_post.append(post_im)
    return out_anno, out_pre, out_post
    
def match_im_label_road(annotations, pre_images, post_images):
    out_pre = []
    out_anno = []
    out_post = []
    for i in annotations:
        tileid = os.path.basename(i).split("roads_speed_")[-1].split(".")[0]
        pre_im = [j for j in pre_images if f"_{tileid}.tif" in j][0]
        post_im = [j for j in post_images if f"_{tileid}.tif" in j][0]
        
        out_anno.append(i)
        out_pre.append(pre_im)
        out_post.append(post_im)
    return out_anno, out_pre, out_post


def geo_coords_to_image_coords(image_geotran, in_wkt):
    """translates WKT geometry in geographic coordinates (latitude, longitude) 
    to WKT geometry in image coordinates (col, row)"""
    xmin = image_geotran[0]
    xres = image_geotran[1]
    ymax = image_geotran[3]
    yres = image_geotran[5]
    
    shapely_poly = shapely.wkt.loads(in_wkt)
    if shapely_poly.geom_type == 'MultiPolygon': # multipolygon
        x, y = [], []
        count = 0
        for polygon in shapely_poly:
            #print(count)
            count+=1
            x1,y1 = polygon.exterior.coords.xy
            x.extend(x1)
            y.extend(y1)
    else: # polygon
        x, y = shapely_poly.exterior.coords.xy
    
    outcoords = [] # [(x. y),(x, y), ...]
    for coord in range(len(x)):\
        outcoords.append(((x[coord]-xmin)/xres, (y[coord]-ymax)/yres))
    out_wkt = Polygon(outcoords).wkt
    return out_wkt

def get_building_features(geojson_dict):
    out_features = []
    for i in geojson_dict["features"]:
        if "building" in i["properties"]:
            if i["properties"]["building"] != None:
                out_features.append(i)
    return out_features

def bldg_geojson_to_wkt(geojsons, images, output_csv_path):
    wkt_list_tot = []
    for i in range(len(images)):
        geojson_path = geojsons[i]
        im_path = images[i]
        ds = gdal.Open(im_path)
        geotran = ds.GetGeoTransform()
        raster_srs = osr.SpatialReference()
        raster_srs.ImportFromWkt(ds.GetProjectionRef())
        raster_srs_epsg = int(raster_srs.GetAttrValue("AUTHORITY", 1))
        assert(raster_srs_epsg == 4326)
        ds = None
        
        im_id = os.path.basename(im_path).split(".")[0]
        
        f = open(geojson_path)
        data = json.load(f)
        f.close()
        
        feats = get_building_features(data)
        if len(feats) == 0:
            wkt_list_tot.append([im_id, "Building", "POLYGON EMPTY", "Null", "Null", "Null"])
        else:
            for b in feats:
                if "flooded" in b["properties"]:
                    flood = b["properties"]["flooded"]
                    fout = True
                    if flood in [None, "no", 0, "0", "No", "false", False, "False"]:
                        fout = False
                    else:
                        pass
                        #print(flood)
                        #print(     b["properties"])
                else:
                    fout = False
                #print(b["geometry"])
                wkt = geo_coords_to_image_coords(geotran, shape(b["geometry"]).wkt)
                wkt_list_tot.append([im_id, "Building", wkt, fout, "Null", "Null"])
            
    
    print(len(wkt_list_tot))
    cols = ['ImageId','Object','Wkt_Pix','Flooded', 'length_m', 'travel_time_s'] 
    df = pd.DataFrame(wkt_list_tot, columns=cols)
    print("df:", df)
    
    df.to_csv(output_csv_path, mode='a', index=False, header=False)
    #df.to_csv(output_csv_path, index=False)

    return df

In [ ]:
root_dir = "" # path to directory holding the aoi_dirs
aoi_dirs = [""] # directory name of each AOI to process
output_csv_path = "" # name of the output csv to write the data to.


# gather the prepped and cleaned geojson labels
geojsons = []
pre_images = []
post_images = []
for i in aoi_dirs:
    assert(os.path.exists(os.path.join(root_dir, i))), "error: aoi doesn't exist"
    anno = glob.glob(os.path.join(root_dir, i, "annotations/prepped_cleaned", "roads_speed*.geojson"))
    pre = glob.glob(os.path.join(root_dir, i, "PRE-event", "*.tif"))
    post = glob.glob(os.path.join(root_dir, i, "POST-event", "*.tif"))
    print("number of annotations: ", len(anno))
    print("number of pre: ", len(pre))
    print("number of post: ", len(post))
    annot, preims, postims = match_im_label_road(anno, pre, post)
    print("AFTER MATCH")
    print("number of annotations: ", len(annot))
    print("number of pre: ", len(preims))
    print("number of post: ", len(postims))
all_records = []
for i in range(len(annot)):
    recs = get_records(annot[i], preims[i])
    if len(recs) == 0:
        all_records.append([os.path.basename(preims[i]).split(".")[0], "Road","LINESTRING EMPTY", "Null", "Null", "Null"])
    for r in recs:
        all_records.append(r)

df = pd.DataFrame(all_records, columns=['ImageId','Object','Wkt_Pix','Flooded', 'length_m', 'travel_time_s'])
print("df:", df)
df.to_csv(output_csv_path, index=False)


# now do the same for buildings.
geojsons = []
pre_images = []
post_images = []
for i in aoi_dirs:
    assert(os.path.exists(os.path.join(root_dir, i))), "error: aoi doesn't exist"

    anno = glob.glob(os.path.join(root_dir, i, "annotations", "*.geojson"))
    pre = glob.glob(os.path.join(root_dir, i, "PRE-event", "*.tif"))
    post = glob.glob(os.path.join(root_dir, i, "POST-event", "*.tif"))

    print("number of annotations: ", len(anno))
    print("number of pre: ", len(pre))
    print("number of post: ", len(post))

    annot, preims, postims = match_im_label(anno, pre, post)
    
    print("AFTER MATCH")
    print("number of annotations: ", len(annot))
    print("number of pre: ", len(preims))
    print("number of post: ", len(postims))

bldg_geojson_to_wkt(annot, preims, output_csv_path)